<a href="https://colab.research.google.com/github/adrianyang2012/landing-ai-webcam/blob/main/landing_ai_with_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
mode = input('are you testing or training')
if mode == 'testing':
  try:
    filename = take_photo(filename=f'testing_pic.png')
    print('Saved to {}'.format(filename))
    Image(filename)
  except Exception as err:
    print(str(err))
else:
  for i in range(0,50):
    try:
      filename = take_photo(filename=f'photo({i}).png')
      print('Saved to {}'.format(filename))
      Image(filename)
    except Exception as err:
      print(str(err))

In [ ]:
import json
import sys
sys.path.append('/content/gdrive/config.ipynb')

# Now we can import the library and use the function.

a = !curl --location --request POST 'https://agw-production.platform.landingai.io/inference/v1/predict?project_id=11402' \
--header 'Content-Type: multipart/form-data' \
--header 'apikey: ............' \
--header 'apisecret: ..................................' \
--form 'file=@"testing_pic.png"'
ans = a[0]


  
 

res = json.loads(ans)
  
print(res)

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread('testing_pic.png',0)
imgplot = plt.imshow(img)

if len(list(res['backbonepredictions'].keys())) != 0:
  
  plt.title(f"labels:{len(list(res['backbonepredictions'].keys()))}")
  for i in range(0,len(list(res['backbonepredictions'].keys()))):
    xmin = res['backbonepredictions'][list(res['backbonepredictions'].keys())[i]]['coordinates']['xmin']
    xmax = res['backbonepredictions'][list(res['backbonepredictions'].keys())[i]]['coordinates']['xmax']
    ymin = res['backbonepredictions'][list(res['backbonepredictions'].keys())[i]]['coordinates']['ymin']
    ymax = res['backbonepredictions'][list(res['backbonepredictions'].keys())[i]]['coordinates']['ymax']
    score = res['backbonepredictions'][list(res['backbonepredictions'].keys())[i]]['score']
    plt.plot([xmin,xmax],[ymin,ymin],color='black')
    plt.plot([xmin,xmax],[ymax,ymax],color='black')
    plt.plot([xmin,xmin],[ymin,ymax],color='black')
    plt.plot([xmax,xmax],[ymin,ymax],color='black')
    label = res['backbonepredictions'][list(res['backbonepredictions'].keys())[i]]['labelName']
    plt.text(xmax,ymax,f'{label} score:{round(score,5)}',color='blue')
else:
  plt.title('nothing to label')
plt.show()